In [8]:

# BMI Calculator 


import tkinter as tk
from tkinter import messagebox
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt

# Database setup
conn = sqlite3.connect("bmi_data.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS bmi_records (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    weight REAL,
    height REAL,
    bmi REAL,
    date TEXT
)
""")
conn.commit()

def calculate_bmi(weight, height):
    return weight / (height ** 2)

def bmi_category(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 24.9:
        return "Normal weight"
    elif 25 <= bmi < 29.9:
        return "Overweight"
    else:
        return "Obese"

def save_data(name, weight, height, bmi):
    date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute(
        "INSERT INTO bmi_records (name, weight, height, bmi, date) VALUES (?, ?, ?, ?, ?)",
        (name, weight, height, bmi, date)
    )
    conn.commit()

def calculate_action():
    try:
        name = entry_name.get()
        weight = float(entry_weight.get())
        height = float(entry_height.get())

        if not name or weight <= 0 or height <= 0:
            raise ValueError

        bmi = calculate_bmi(weight, height)
        category = bmi_category(bmi)

        label_result.config(text=f"BMI: {bmi:.2f}\nCategory: {category}")
        save_data(name, weight, height, bmi)

    except:
        messagebox.showerror("Error", "Please enter valid inputs")

def show_history():
    cursor.execute("SELECT date, bmi FROM bmi_records")
    data = cursor.fetchall()

    if not data:
        messagebox.showinfo("Info", "No data available")
        return

    dates = [row[0] for row in data]
    bmis = [row[1] for row in data]

    plt.figure()
    plt.plot(dates, bmis, marker='o')
    plt.xticks(rotation=45)
    plt.xlabel("Date")
    plt.ylabel("BMI")
    plt.title("BMI Trend")
    plt.tight_layout()
    plt.show()

# GUI setup
root = tk.Tk()
root.title("BMI Calculator")

tk.Label(root, text="Name").grid(row=0, column=0)
tk.Label(root, text="Weight (kg)").grid(row=1, column=0)
tk.Label(root, text="Height (m)").grid(row=2, column=0)

entry_name = tk.Entry(root)
entry_weight = tk.Entry(root)
entry_height = tk.Entry(root)

entry_name.grid(row=0, column=1)
entry_weight.grid(row=1, column=1)
entry_height.grid(row=2, column=1)

tk.Button(root, text="Calculate BMI", command=calculate_action).grid(row=3, column=0, columnspan=2)
tk.Button(root, text="Show BMI History", command=show_history).grid(row=4, column=0, columnspan=2)

label_result = tk.Label(root, text="")
label_result.grid(row=5, column=0, columnspan=2)

root.mainloop()
